# Data Preparation

## Import Modules

In [ ]:
import os
import pandas as pd
import duckdb
import numpy as np

```bash
cd data
uvx --from deweypy dewey --api-key <API_KEY> speedy-download <FOLDER_ID>
```

## Constants

In [ ]:
# San Diego Country Zip Codes 
zipcodes = [
    91901, 91902, 91903, 91905, 91906, 91908, 91909, 91910, 91911, 91913,
    91914, 91915, 91916, 91917, 91919, 91921, 91931, 91932, 91934, 91935,
    91941, 91942, 91943, 91944, 91945, 91946, 91947, 91948, 91950, 91951,
    91962, 91963, 91964, 91965, 91976, 91977, 91978, 91979, 91980, 91987,
    92003, 92004, 92007, 92008, 92009, 92010, 92011, 92013, 92014, 92018,
    92019, 92020, 92021, 92022, 92023, 92024, 92025, 92026, 92027, 92028,
    92029, 92033, 92036, 92037, 92038, 92039, 92040, 92046, 92049, 92051,
    92052, 92054, 92055, 92056, 92057, 92058, 92059, 92060, 92061, 92062,
    92064, 92065, 92066, 92067, 92068, 92069, 92070, 92071, 92072, 92074,
    92075, 92078, 92079, 92082, 92083, 92084, 92085, 92086, 92088, 92090,
    92091, 92092, 92093, 92096, 92099, 92101, 92102, 92103, 92104, 92105,
    92106, 92107, 92108, 92109, 92110, 92111, 92112, 92113, 92114, 92115,
    92116, 92117, 92118, 92119, 92120, 92121, 92122, 92123, 92124, 92126,
    92127, 92128, 92129, 92130, 92131, 92132, 92133, 92134, 92135, 92136,
    92137, 92138, 92139, 92140, 92142, 92143, 92144, 92145, 92147, 92149,
    92150, 92152, 92153, 92154, 92155, 92158, 92159, 92160, 92161, 92162,
    92163, 92164, 92165, 92166, 92167, 92168, 92169, 92170, 92171, 92172,
    92173, 92174, 92175, 92176, 92177, 92178, 92179, 92180, 92182, 92184,
    92186, 92187, 92190, 92191, 92192, 92193, 92194, 92195, 92196, 92197,
    92198, 92199
]

GLOBAL_PLACES_DIR = 'data/dewey-downloads/global-places-for-lat-long-ca' # adjust as needed
SPEND_PATTERNS_DIR = 'data/dewey-downloads/safegraph-spend-patterns-ca-only'

In [ ]:
# Load the Global Places dataset
GLOBAL_PLACES_PATH = os.path.join(GLOBAL_PLACES_DIR, '*.parquet')
con = duckdb.connect()

# Filter for only San Diego County zip codes (removed columns and filtered to CA in Dewey)
df = con.execute(f"""
            SELECT
                *
            FROM '{GLOBAL_PLACES_PATH}' 
            WHERE POSTAL_CODE IN ({','.join(map(str, zipcodes))})""").fetchdf()
df.head()

In [ ]:
# export subset to parquet
df.to_parquet('data/san-diego-county-places.parquet')

In [ ]:
df.columns

In [ ]:
# np.sort(df['TOP_CATEGORY'].unique()) # remove category as its available in spend

In [ ]:
df['POSTAL_CODE'].value_counts()

## Spend Patterns

In [ ]:
SPEND_PATTERNS_PATH = os.path.join(SPEND_PATTERNS_DIR, '2025-07--data_01bffb7b-0106-c81a-0042-fa0703e58316_108_2_0.snappy.parquet')

spend_df = con.execute(f"""
            SELECT 
                *
            FROM '{SPEND_PATTERNS_PATH}'
            WHERE POSTAL_CODE IN ({','.join(map(str, zipcodes))}) AND
            BRANDS IS NOT NULL""").fetchdf()
spend_df.head()

In [ ]:
spend_df.to_parquet('data/san-diego-county-spend-patterns.parquet')

In [ ]:
con.execute("CREATE OR REPLACE TABLE places AS SELECT * FROM 'data/san-diego-county-places.parquet'")
con.execute("CREATE OR REPLACE TABLE spend AS SELECT * FROM 'data/san-diego-county-spend-patterns.parquet'")

joined_df = con.execute("""
    SELECT 
        pl.*,
        sp.*
    FROM places pl
    JOIN spend sp
      ON pl.PLACEKEY = sp.PLACEKEY
""").fetchdf()

joined_df.head()


In [ ]:
# show duplicate columns
end_1_cols = [col for col in joined_df.columns if col.endswith('_1')]
print(end_1_cols)

In [ ]:
# drop duplicate columns
joined_df = joined_df.loc[:, ~joined_df.columns.str.endswith('_1')]


In [ ]:
joined_df.to_parquet('data/san-diego-county-places-spend.parquet')
